### Allen function

#### Arguments
##### The allen function allows for the selection of tracer experiments based upon primary injections sites and retrograde tracer activity. The functions accepts a number of arguments:

- ROI_anterograde: ROI_anterograde has to be a ROI derived from 'http://api.brain-map.org/api/v2/structure_graph_download/1.json'. All experiments with primary injection  sites in that ROI will be exported as nifti file                             
- ROI_retrograde: ROI_anterograde has to be a ROI derived from 'http://api.brain-map.org/api/v2/structure_graph_download/1.json'. All experiments in which the tracer activity surpassed a treshold in that ROI will be exported as a nifti file     
                             
- data: The wild-tracer which should be downloaded via the 'allen_download' function. The data should be imported in python as numpy array
- activity_treshold: The activity_treshold specifies the amount of tracer-activity single voxels have to have in order to count towards the area treshold. The activity_treshold can be any value between 0 and 1 (default = 0.2)
- area_treshold: The area_treshold specifies the fraction of voxels of ROI that have to match the activity treshold in order for ROI to be considered to have significant tracer activity (and hence be included into the ROI_retrograde tracer nifti file). When using larger ROI's (for example the isocortex), you definitely want to lower the area_treshold. The area_treshold can be any value between 0 and 1 (default = 0.05)
- mask: Specifies whether a binary mask should be created for the ROI's (default=False)
- is_injection: Determines whether selected experiments for the retrograde tracer should include experiments in which the primary injection site is located in ROI itself (default=False)
- directory: The directory in which the output will be exported (default=current directory)
- resolution The resolution of the tracer-data. This should match the resolution of the downloaded tracer data. I strongly recommend to keep it at 100 muM, since the function will require much more memory and time to run if the resolution is increased

### Data
- the data can be loaded into python with the following code
- import numpy as np
- import nibabel as nib
- import pandas as pd
- import os
- data= nib.load('tracer_data_masked.nii.gz')
- data = np.array(data.dataobj)

#### Examples
#####
- allen(): displays a help message
- allen('Infralimbic area', None, data): creates a nifti file for all the experiments with a primary injection site in the infralimbic area
- allen(None, 'Prelimbic area', data, mask=True): creates a nifti file for all the experiments with tresholded tracer activity in the  prelimbic area (not including experiments in which there was a primary injection site in the prelimbic area) and also creates a mask
- allen (None, 'Prelimbic area', data, mask=True, is_injection=True): the same as before, but now also includes experiments in which there was a primary injection site in the region
- allen (None, 'Secondary motor area', data, activity_treshold=0.3): creates a nifti file for all the experiments with tresholded tracer activity in the secondary motor area at an activity_treshold of 0.3
- allen (None, 'Secondary motor area',data, area_treshold=0.1): creates a nifti file for all the experiments with tresholded tracer activity in the secondary motor area at an area treshold of 0.1
- allen ('Anterior cingulate area', 'Anterior cingulate area', data, directory= 'home/example'): creates two nifti files, one with all the tracer data with a primary injection site in the anterior cingulate area, and one with all experiments with tresholded tracer activity in the anterior cingulate area (not including experiments in which there was a primary injection site in the anterior cingulate area). And exports the nifti files to a custom directory


In [1]:
#installing packages
import numpy as np
import nibabel as nib
import pandas as pd
import os

In [2]:
#loading the data, check directory
os.chdir('/project/2421074.01/Allen_function/')
data= nib.load('tracer_data.nii.gz')
data = np.array(data.dataobj)

In [3]:


def allen (ROI_anterograde=None, ROI_retrograde=None, data=None, activity_treshold=0.2, area_treshold=0.05, mask=False, is_injection=False, directory=None, resolution=100 ):
    
      #INFORMATION MESSAGE

    if( (ROI_anterograde == None) and (ROI_retrograde == None) and (data ==None)) :
        print ('allen takes the following arguments:')
        print ('')
        print ('ROI_anterograde:                 The anterograde ROI, all experiments with primary injection sites in this region')
        print ('                                 will be exported as a nifti file')
        print ('ROI_retrograde:                  The  retrograde ROI, all experiments where the tracer-activity in this region')
        print ('                                 surpasses a treshold wil be exported as a nifti file')
        print ('data:                            The wild type tracer-data, which can be downloaded from the from')
        print ('                                 the allen brain data base, the data should be a numpy array loaded into python')   
        print ('activity_treshold:               The activity treshold specifies the amount of tracer-activity single voxels have to')
        print ('                                 have in order to count towards the area_treshold (default=0.2)') 
        print ('area_treshold:                   The area treshold specifies the fraction of voxels of a ROI that have to match the')
        print ('                                 activity_treshold in order for a ROI to to be considered to have significant tracer')
        print ('                                 activity (default=0.05)')
        print ("mask                             Specifies whether a  binary mask should be created and exported as nifti file for")
        print ("                                 the selected ROI's (default=false)")
        print ("is_injection                     Determines whether experiments for the retrograde tracer should include experiments")
        print ("                                 in which the tracer is injected in the ROI itself (default=False)")
        print ("directory                        Set the directory to where the output will be export, (default= current directory")
        print ("resolution                       Set the resolution (default=100). The resolution should match the resolution of")
        print ("                                 the downloaded tracer data. Moreover, I strongly recommend working with the 100 muM")
        print ("                                 since the programm will slow down by a huge amount if the resolution is increased")
        print ('')
        print ('please contact Aran van Hout at aranvanhout@hotmail.com for any questions regarding the allen function')
        return
    
    #GENERAL DATA NEEDED FOR BOTH THE ANTEROGRADE AND RETROGRADE FUNCTION
    
    #get the size of the data, display error message when no data is selected
    try:
        (x,y,z,e)=data.shape#code this differently
    except AttributeError:
        print('-no data selected')
        return 
    
    #import the mouse connectivity cache at specified resolution
    from allensdk.core.mouse_connectivity_cache import MouseConnectivityCache
    mcc = MouseConnectivityCache(resolution=resolution)
    
    #import the structure tree    
    structure_tree = mcc.get_structure_tree()
    
    #import all the experiments IDs of the wild-type data
    df = pd.read_csv (r'WT_experiment_id_list.csv')
    complete_id_list = df['id'].tolist()
    
    #import mask toolbox
    from allensdk.core.reference_space_cache import ReferenceSpaceCache
    reference_space_key = 'annotation/ccf_2017'
    rspc = ReferenceSpaceCache(resolution, reference_space_key, manifest='manifest.json')
    rsp = rspc.get_reference_space()
    tree = rspc.get_structure_tree(structure_graph_id=1) 
     
    #DIRECTORY
    import os
    if directory==None:
        output_dir = os.getcwd()
    else:
        output_dir=directory
        
    #UPDATE CHECK
    
    #download the (up-to-date) experiment id file from allen
    mcc_check=mcc.get_experiments(cre=False)
    mcc_check=pd.DataFrame(mcc_check)
    mcc_check=mcc_check['id'].to_list()
    
    
    #compare the two lists
    if mcc_check != complete_id_list:
        print ("The downloaded tracer-data/projection_search_results is no longer up-to-date!")
        print ("Please download the tracer-data and the projection search results again")
        print ("Contact Aran van Hout at aranvanhout@hotmail.com for further questions")
        print ("")
        print ("allen ended")
        return
    #Checks whether the downloaded data has the same number of experiments as the experiment id list
    if e != len(mcc_check) or e!= len(complete_id_list):
        print ("The downloaded tracer-data/projection_search_results is no longer up-to-date!")
        print ("Please download the tracer-data and the projection search results again")
        print ("Contact Aran van Hout at aranvanhout@hotmail.com for further questions")
        print ("")
        print ("allen ended")
        return
    
    
    
    #ANTEROGRADE FUNCTION
    
    if ROI_anterograde!=None:
        #check whether the region exists, if so get the tracer experiments
        try:        
            #get the tracer experiments
            ROI_anterograde_ID=structure_tree.get_structures_by_name([ROI_anterograde])[0]['id']#get the structure id
            ROI_anterograde_experiments=mcc.get_experiments(cre=False,
                                                injection_structure_ids=[ROI_anterograde_ID]) #get all experimets with an injection in the ROI
        except KeyError:
            print ("'{}' -is not a existing region, see http://api.brain-map.org/api/v2/structure_graph_download/1.json".format(ROI_anterograde))
            print ('for a list of all the regions')
            print ("")
            print ('allen ended')
            return;
        
        #if there are no experiments with a primary injection in selected region, display error message
        if len (ROI_anterograde_experiments)==0:
            print ("-there are no experiments with a primary injection in the '{}'".format(ROI_anterograde))
            
    
        #if there experiments with primary injection sites in the region, create a nifti file with these experiments
        
        if len(ROI_anterograde_experiments)>0:
                   
            #create a list of all the experiments IDs
            df=pd.DataFrame(ROI_anterograde_experiments)
            ROI_anterograde_experiments_list=df['id'].to_list()
        
            #convert the list to indices, related to the downloaded tracer data
            ROI_anterograde_experiment_index=[]
            for experiment_id in ROI_anterograde_experiments_list:
                experiment_index = complete_id_list.index(experiment_id) 
                ROI_anterograde_experiment_index.append(experiment_index)
    
            #create empty matrix to store the experiments in
            merge_matrix = np.zeros(x*y*z*len(ROI_anterograde_experiment_index))
            merge_matrix = np.reshape(merge_matrix,(x,y,z,len(ROI_anterograde_experiment_index)))

            #store the experiments
            a=0
            for anterograde_tracer_id in ROI_anterograde_experiment_index:
                tracer= data[:,:,:,anterograde_tracer_id]
                merge_matrix[:,:,:,a]=tracer
                a=a+1                                                   
                           
            #export array as nifti and save
            tracer_nifti= nib.Nifti1Image(merge_matrix, affine=np.eye(4))
            nib.save(tracer_nifti, os.path.join(output_dir, 'anterograde_tracer_{}.nii.gz'.format(ROI_anterograde)))#would be very cool to have name of the roi   
            print ("-nifti file with the '{}' as ROI for anterograde tracer succesfully created and exported ({} experiments in total)".format(ROI_anterograde, len(ROI_anterograde_experiment_index)))
            print (" to '{}'.".format(output_dir))
            
        if mask==True:
            Mask_ID=tree.get_structures_by_name([ROI_anterograde])[0]['id']
            Mask=rsp.make_structure_mask([Mask_ID])
            i=np.where(Mask==1)#check the mask actually has values
            if len(i[0])==0:
                n=tree.parents([ROI_anterograde_ID])
                n=n[0]
                n=n['name']
                print ("-it is not possible to create a mask for '{}'.This is probably due to high locational specifity of the ROI".format(ROI_anterograde))
                if n!='root':
                    print (" please select a larger ROI, for example the '{}'".format(n))
                else:
                    print ('please select a larger ROI')
            
            else:
                mask_nifti= nib.Nifti1Image(Mask, affine=np.eye(4))
                nib.save(mask_nifti, os.path.join(output_dir, 'mask_{}.nii.gz'.format(ROI_anterograde))) 
                print ("-mask for the '{}' succesfully created and exported to '{}'".format(ROI_anterograde, output_dir))
               
                
    #RETROGRADE FUNCTION
    if ROI_retrograde!=None:
        #check whether the region exists, if so get the tracer experiments
        try:        
            #get the tracer experiments
            ROI_retrograde_ID=structure_tree.get_structures_by_name([ROI_retrograde])[0]['id']#get the structure id
        except KeyError:
            print ("")
            print ("-'{}' is not a existing region, see http://api.brain-map.org/api/v2/structure_graph_download/1.json".format(ROI_retrograde))
            print ('for a list of all the regions')
            print ("")
            print ('allen ended')
            return;        
                               
        #create a mask
        Mask_ID=tree.get_structures_by_name([ROI_retrograde])[0]['id']
        Mask=rsp.make_structure_mask([Mask_ID])
        i=np.where(Mask==1)#check the mask actually has values
        if len(i[0])==0:
            n=tree.parents([ROI_retrograde_ID])
            n=n[0]
            n=n['name']
            print('')
            print ("-it is not possible to select '{}' as a retrograde ROI.This is probably due to the high locational specifity of the ROI".format(ROI_retrograde))
            if n!='root':
                print (" please select a larger ROI, for example the '{}'".format(n))
            else:
                print ('please select a larger retrograde ROI')    
            return
       
       #sets the area treshold_corrected, based upon the size of the mask
        single_experiment=data[:,:,:,0]
        single_experiment=single_experiment[Mask ==1]#get only the activity of the masked region
        area_treshold_corrected = len(single_experiment)*area_treshold#determines the fraction of the area 

        #create a lists of the scans that have tresholded activity in the mask
        ROI_retrograde_experiment_list=[]
        
        for experiment in range(e):
            single_experiment=data[:,:,:,experiment]
            single_experiment=single_experiment[Mask ==1]#get only the activity of the masked region               
            single_experiment=single_experiment[single_experiment>activity_treshold]#determines the amount of voxels above the activity treshold
            if len(single_experiment) >= area_treshold_corrected:#determines whether the amount of voxels above the activity treshold is higher than the area_treshold_corrected
                ROI_retrograde_experiment_list.append(experiment)
        
        #remove experiments in which there was an injection in the ROI_retrograde
        if is_injection==False:
            ROI_pi_retrograde_experiments=mcc.get_experiments(cre=False,
                            injection_structure_ids=[ROI_retrograde_ID]) #get all experimets with an injection in the ROI
            if len(ROI_pi_retrograde_experiments)>0: #only when there are ROIs with a primary injection in the region, duplicates have to be removed
                #create a list of all the experiments IDs
                df=pd.DataFrame(ROI_pi_retrograde_experiments)
                ROI_pi_retrograde_experiments_list=df['id'].to_list()
                
                #convert the list to indices, related to the downloaded tracer data
                ROI_pi_retrograde_experiment_index=[]
                for experiment_id in ROI_pi_retrograde_experiments_list:
                    experiment_index = complete_id_list.index(experiment_id) 
                    ROI_pi_retrograde_experiment_index.append(experiment_index)
                    
                #remove the duplicates
                no_duplicates=[]#this code is not very concise, however the remove method did not work properly
                for element in ROI_retrograde_experiment_list:
                    if element not in ROI_pi_retrograde_experiment_index:
                        no_duplicates.append(element) 
                        ROI_retrograde_experiment_list=no_duplicates  
                
        
         
        if len(ROI_retrograde_experiment_list)==0:
            print ("no experiments with significant tracer activity in the '{}', try adjusting the treshold".format(ROI_retrograde))
            print ("via 'area_treshold' or 'activity_treshold' or select another ROI")
            if mask==True & (ROI_retrograde!= ROI_anterograde):#ensuring that a mask will still be created
                mask_nifti= nib.Nifti1Image(Mask, affine=np.eye(4))
                nib.save(mask_nifti, os.path.join(output_dir, 'mask_{}.nii.gz'.format(ROI_retrograde)))#would be very cool to have name of the roi   
                print ("-mask for the '{}' succesfully created and exported to '{}'".format(ROI_retrograde, output_dir))
                return
            return
                
        #add scans from this list to a new numpy array 
        merge_matrix = np.zeros(x*y*z*len(ROI_retrograde_experiment_list))
        merge_matrix = np.reshape(merge_matrix,(x,y,z,len(ROI_retrograde_experiment_list)))

        #store the experiments
        a=0
        for retrograde_tracer_id in ROI_retrograde_experiment_list:
            tracer= data[:,:,:,retrograde_tracer_id]
            merge_matrix[:,:,:,a]=tracer
            a=a+1
        
            
        
        #export array as nifti and save
        tracer_nifti= nib.Nifti1Image(merge_matrix, affine=np.eye(4))
        nib.save(tracer_nifti,os.path.join(output_dir, 'retrograde_tracer_{}.nii.gz'.format(ROI_retrograde)))#would be very cool to have name of the roi
        print ("")
        print ("-nifti file with the '{}' as ROI for retrograde tracer succesfully created and exported ({} experiments in total)".format(ROI_retrograde, len(ROI_retrograde_experiment_list)))
        print (" to '{}'.".format(output_dir))
        if mask==True & (ROI_retrograde != ROI_anterograde):
            mask_nifti= nib.Nifti1Image(Mask, affine=np.eye(4))
            nib.save(mask_nifti,os.path.join(output_dir, 'mask_{}.nii.gz'.format(ROI_retrograde)))#would be very cool to have name of the roi   
            print ("-mask for the '{}' succesfully created and exported to '{}'".format(ROI_retrograde, output_dir))
    
    return;

            

In [52]:
#example code
allen(ROI_anterograde=None, ROI_retrograde= 'Secondary motor area', data=data,activity_treshold=0.15, area_treshold=0.2, mask=True, is_injection=False, directory='/project/2421074.01/Allen_function/demonstration')


-nifti file with the 'Secondary motor area' as ROI for retrograde tracer succesfully created and exported (8 experiments in total)
 to '/project/2421074.01/Allen_function/demonstration'.
-mask for the 'Secondary motor area' succesfully created and exported to '/project/2421074.01/Allen_function/demonstration'
